<a href="https://colab.research.google.com/github/MatheusHenriqueDaSilva/data_science/blob/main/pro_acoes_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import joblib
from sklearn.model_selection import train_test_split
import yaml
##!pip install pandas pyarrow fastparquet

In [ ]:
tamanho_campos=[2,8,2,12,3,12,10,3,4,13,13,13,13,13,13,13,5,18,18,13,1,8,7,13,12,3]
ANOS=[2018,2019,2020,2021,2022,2023,2024]
dados = {}  # Dicionário para armazenar os DataFrames

for ano in ANOS:
  dados[f'data_{ano}'] = pd.read_fwf(f'/content/COTAHIST_A{ano}.TXT', widths=tamanho_campos, header=0)

  ## Nomear as colunas

  dados[f'data_{ano}'].columns = [
  "tipo_registro",
  "data_pregao",
  "cod_bdi",
  "cod_negociacao",
  "tipo_mercado",
  "noma_empresa",
  "especificacao_papel",
  "prazo_dias_merc_termo",
  "moeda_referencia",
  "preco_abertura",
  "preco_maximo",
  "preco_minimo",
  "preco_medio",
  "preco_ultimo_negocio",
  "preco_melhor_oferta_compra",
  "preco_melhor_oferta_venda",
  "numero_negocios",
  "quantidade_papeis_negociados",
  "volume_total_negociado",
  "preco_exercicio",
  "ìndicador_correcao_precos",
  "data_vencimento" ,
  "fator_cotacao",
  "preco_exercicio_pontos",
  "codigo_isin",
  "num_distribuicao_papel"]

  # Eliminar a última linha
  linha=len(dados[f'data_{ano}']["data_pregao"])
  dados[f'data_{ano}']=dados[f'data_{ano}'].drop(linha-1)

  # Ajustar valores com virgula (dividir os valores dessas colunas por 100)
  listaVirgula=[
  "preco_abertura",
  "preco_maximo",
  "preco_minimo",
  "preco_medio",
  "preco_ultimo_negocio",
  "preco_melhor_oferta_compra",
  "preco_melhor_oferta_venda",
  "volume_total_negociado",
  "preco_exercicio",
  "preco_exercicio_pontos"
  ]

  for coluna in listaVirgula:
      dados[f'data_{ano}'][coluna]=[i/100. for i in dados[f'data_{ano}'][coluna]]

  dados[f'data_{ano}'] = dados[f'data_{ano}'][(dados[f'data_{ano}']['tipo_mercado']==10) & (dados[f'data_{ano}']['cod_negociacao'] == 'ITUB4')]


In [ ]:
data_itub4_save =  pd.concat([dados[f'data_2018'],dados[f'data_2019'],dados[f'data_2020'],dados[f'data_2021'],dados[f'data_2022'],dados[f'data_2023'],dados[f'data_2024']], axis = 0).reset_index()
data_itub4_save.to_parquet("data_itub4.parquet", engine="pyarrow", index=False)


,index,tipo_registro,data_pregao,cod_bdi,cod_negociacao,tipo_mercado,noma_empresa,especificacao_papel,prazo_dias_merc_termo,moeda_referencia,...,numero_negocios,quantidade_papeis_negociados,volume_total_negociado,preco_exercicio,ìndicador_correcao_precos,data_vencimento,fator_cotacao,preco_exercicio_pontos,codigo_isin,num_distribuicao_papel
0,1016,1,20180102,2.0,ITUB4,10,ITAUUNIBANCO,PN ED N1,NaN,R$,...,30810.0,11100600.0,483649086.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,232.0
1,3099,1,20180103,2.0,ITUB4,10,ITAUUNIBANCO,PN ED N1,NaN,R$,...,18377.0,14863800.0,654269523.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,232.0
2,5397,1,20180104,2.0,ITUB4,10,ITAUUNIBANCO,PN ED N1,NaN,R$,...,28294.0,15481700.0,696661801.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,232.0
3,7763,1,20180105,2.0,ITUB4,10,ITAUUNIBANCO,PN ED N1,NaN,R$,...,19882.0,10478300.0,472439105.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,232.0
4,10039,1,20180108,2.0,ITUB4,10,ITAUUNIBANCO,PN ED N1,NaN,R$,...,20228.0,8472100.0,381831069.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,232.0


In [3]:
data_itub4 = pd.read_parquet("data_itub4.parquet", engine="pyarrow").reset_index(drop=True)
data_itub4.head()

,index,tipo_registro,data_pregao,cod_bdi,cod_negociacao,tipo_mercado,noma_empresa,especificacao_papel,prazo_dias_merc_termo,moeda_referencia,...,numero_negocios,quantidade_papeis_negociados,volume_total_negociado,preco_exercicio,ìndicador_correcao_precos,data_vencimento,fator_cotacao,preco_exercicio_pontos,codigo_isin,num_distribuicao_papel
0,1016,1,20180102,2.0,ITUB4,10,ITAUUNIBANCO,PN ED N1,NaN,R$,...,30810.0,11100600.0,483649086.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,232.0
1,3099,1,20180103,2.0,ITUB4,10,ITAUUNIBANCO,PN ED N1,NaN,R$,...,18377.0,14863800.0,654269523.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,232.0
2,5397,1,20180104,2.0,ITUB4,10,ITAUUNIBANCO,PN ED N1,NaN,R$,...,28294.0,15481700.0,696661801.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,232.0
3,7763,1,20180105,2.0,ITUB4,10,ITAUUNIBANCO,PN ED N1,NaN,R$,...,19882.0,10478300.0,472439105.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,232.0
4,10039,1,20180108,2.0,ITUB4,10,ITAUUNIBANCO,PN ED N1,NaN,R$,...,20228.0,8472100.0,381831069.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,232.0


In [ ]:
data_itub4.tail(
)

,index,tipo_registro,data_pregao,cod_bdi,cod_negociacao,tipo_mercado,noma_empresa,especificacao_papel,prazo_dias_merc_termo,moeda_referencia,...,numero_negocios,quantidade_papeis_negociados,volume_total_negociado,preco_exercicio,ìndicador_correcao_precos,data_vencimento,fator_cotacao,preco_exercicio_pontos,codigo_isin,num_distribuicao_papel
1681,1347607,1,20241007,2.0,ITUB4,10,ITAUUNIBANCO,PN EJ N1,NaN,R$,...,22970.0,23310400.0,818620449.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,342.0
1682,1349245,1,20241008,2.0,ITUB4,10,ITAUUNIBANCO,PN EJ N1,NaN,R$,...,33539.0,23103400.0,817031494.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,342.0
1683,1351069,1,20241009,2.0,ITUB4,10,ITAUUNIBANCO,PN EJ N1,NaN,R$,...,35673.0,20924100.0,729268413.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,342.0
1684,1352647,1,20241010,2.0,ITUB4,10,ITAUUNIBANCO,PN EJ N1,NaN,R$,...,27883.0,20487100.0,713223496.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,342.0
1685,1354390,1,20241011,2.0,ITUB4,10,ITAUUNIBANCO,PN N1,NaN,R$,...,29848.0,16965500.0,588880130.0,0.0,0.0,99991231.0,1.0,0.0,BRITUBACNPR1,342.0


In [4]:
def generate_metadata(dataframe):

    metadata2 = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata2=metadata2.sort_values(by='tipo')
    metadata2 = metadata2.reset_index(drop=True)

    return metadata2

metadados = generate_metadata(data_itub4)
metadados.head(50)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,index,int64,0,0.0,1686
1,tipo_registro,int64,0,0.0,1
2,tipo_mercado,int64,0,0.0,1
3,preco_exercicio_pontos,float64,0,0.0,1
4,fator_cotacao,float64,0,0.0,1
5,data_vencimento,float64,0,0.0,1
6,ìndicador_correcao_precos,float64,0,0.0,1
7,preco_exercicio,float64,0,0.0,1
8,volume_total_negociado,float64,0,0.0,1686
9,quantidade_papeis_negociados,float64,0,0.0,1682


# Nova seção

In [5]:
lista_spec = ['quantidade_papeis_negociados','numero_negocios','preco_melhor_oferta_venda','preco_melhor_oferta_compra',
              'preco_medio','preco_minimo','preco_maximo','preco_abertura','num_distribuicao_papel','preco_ultimo_negocio']

data_itub4_clean = data_itub4.loc[:,lista_spec]
data_itub4_clean.shape



(1686, 10)

In [6]:

for i in lista_spec:
  data_itub4_clean[f'max20_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=20).max().fillna(0)
  data_itub4_clean[f'max60_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=60).max().fillna(0)
  data_itub4_clean[f'max120_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=120).max().fillna(0)
  data_itub4_clean[f'max240_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=240).max().fillna(0)
  data_itub4_clean[f'max480_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=480).max().fillna(0)
  data_itub4_clean[f'max720_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=720).max().fillna(0)

  data_itub4_clean[f'min20_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=20).min().fillna(0)
  data_itub4_clean[f'min60_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=60).min().fillna(0)
  data_itub4_clean[f'min120_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=120).min().fillna(0)
  data_itub4_clean[f'min240_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=240).min().fillna(0)
  data_itub4_clean[f'min480_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=480).min().fillna(0)
  data_itub4_clean[f'min720_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=720).min().fillna(0)

  data_itub4_clean[f'media20_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=20).mean().fillna(0)
  data_itub4_clean[f'media60_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=60).mean().fillna(0)
  data_itub4_clean[f'media120_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=120).mean().fillna(0)
  data_itub4_clean[f'media240_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=240).mean().fillna(0)
  data_itub4_clean[f'media480_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=480).mean().fillna(0)
  data_itub4_clean[f'media720_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=720).mean().fillna(0)

data_itub4_clean2 = data_itub4_clean.iloc[721:,:].reset_index(drop=True)

for i in lista_spec:
  data_itub4_clean2[f'razmax20_60_{i}'] = round(data_itub4_clean2[f'max20_{i}'],4)/round(data_itub4_clean2[f'max60_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmax60_120_{i}'] = round(data_itub4_clean2[f'max60_{i}'],4)/round(data_itub4_clean2[f'max120_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmax120_240_{i}'] = round(data_itub4_clean2[f'max120_{i}'],4)/round(data_itub4_clean2[f'max240_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmax240_480_{i}'] = round(data_itub4_clean2[f'max240_{i}'],4)/round(data_itub4_clean2[f'max480_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmax20_120_{i}'] = round(data_itub4_clean2[f'max20_{i}'],4)/round(data_itub4_clean2[f'max120_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmax60_240{i}'] = round(data_itub4_clean2[f'max60_{i}'],4)/round(data_itub4_clean2[f'max240_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmax120_480{i}'] = round(data_itub4_clean2[f'max120_{i}'],4)/round(data_itub4_clean2[f'max480_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmax20_240{i}'] = round(data_itub4_clean2[f'max20_{i}'],4)/round(data_itub4_clean2[f'max240_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmax60_480{i}'] = round(data_itub4_clean2[f'max60_{i}'],4)/round(data_itub4_clean2[f'max480_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmax20_480{i}'] = round(data_itub4_clean2[f'max20_{i}'],4)/round(data_itub4_clean2[f'max480_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmin20_60_{i}'] = round(data_itub4_clean2[f'min20_{i}'],4)/round(data_itub4_clean2[f'min60_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmin60_120_{i}'] = round(data_itub4_clean2[f'min60_{i}'],4)/round(data_itub4_clean2[f'min120_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmin120_240_{i}'] = round(data_itub4_clean2[f'min120_{i}'],4)/round(data_itub4_clean2[f'min240_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmin240_480_{i}'] = round(data_itub4_clean2[f'min240_{i}'],4)/round(data_itub4_clean2[f'min480_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmin20_120_{i}'] = round(data_itub4_clean2[f'min20_{i}'],4)/round(data_itub4_clean2[f'min120_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmin60_240{i}'] = round(data_itub4_clean2[f'min60_{i}'],4)/round(data_itub4_clean2[f'min240_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmin120_480{i}'] = round(data_itub4_clean2[f'min120_{i}'],4)/round(data_itub4_clean2[f'min480_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmin20_240{i}'] = round(data_itub4_clean2[f'min20_{i}'],4)/round(data_itub4_clean2[f'min240_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmin60_480{i}'] = round(data_itub4_clean2[f'min60_{i}'],4)/round(data_itub4_clean2[f'min480_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmin20_480{i}'] = round(data_itub4_clean2[f'min20_{i}'],4)/round(data_itub4_clean2[f'min480_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmedia20_60_{i}'] = round(data_itub4_clean2[f'media20_{i}'],4)/round(data_itub4_clean2[f'media60_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmedia60_120_{i}'] = round(data_itub4_clean2[f'media60_{i}'],4)/round(data_itub4_clean2[f'media120_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmedia120_240_{i}'] = round(data_itub4_clean2[f'media120_{i}'],4)/round(data_itub4_clean2[f'media240_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmedia240_480_{i}'] = round(data_itub4_clean2[f'media240_{i}'],4)/round(data_itub4_clean2[f'media480_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmedia20_120_{i}'] = round(data_itub4_clean2[f'media20_{i}'],4)/round(data_itub4_clean2[f'media120_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmedia60_240{i}'] = round(data_itub4_clean2[f'media60_{i}'],4)/round(data_itub4_clean2[f'media240_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmedia120_480{i}'] = round(data_itub4_clean2[f'media120_{i}'],4)/round(data_itub4_clean2[f'media480_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmedia20_240{i}'] = round(data_itub4_clean2[f'media20_{i}'],4)/round(data_itub4_clean2[f'media240_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmedia60_480{i}'] = round(data_itub4_clean2[f'media60_{i}'],4)/round(data_itub4_clean2[f'media480_{i}'].fillna(0),4)
  data_itub4_clean2[f'razmedia20_480{i}'] = round(data_itub4_clean2[f'media20_{i}'],4)/round(data_itub4_clean2[f'media480_{i}'].fillna(0),4)





<ipython-input-6-6af5b3ae1ac6>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_itub4_clean[f'min240_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=240).min().fillna(0)
<ipython-input-6-6af5b3ae1ac6>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_itub4_clean[f'min480_{i}'] = data_itub4_clean[f'{i}'].shift(0).rolling(window=480).min().fillna(0)
<ipython-input-6-6af5b3ae1ac6>:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has p

In [7]:
#data_itub4['var_preco_ultimo_negocio'] = data_itub4['preco_ultimo_negocio'].pct_change(-5).fillna(0)
data_itub4_clean2['target']  = data_itub4_clean2['preco_ultimo_negocio'].shift(-20)
# lista_anal = ['quantidade_papeis_negociados','numero_negocios','preco_melhor_oferta_venda','preco_melhor_oferta_compra',
#               'preco_medio','preco_minimo','preco_maximo','preco_abertura','num_distribuicao_papel','preco_ultimo_negocio','target']
data_itub4_clean3 = data_itub4_clean2.iloc[:-20,:].reset_index(drop=True)

data_itub4_clean3.shape

<ipython-input-7-aa42e6dd667c>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_itub4_clean2['target']  = data_itub4_clean2['preco_ultimo_negocio'].shift(-20)


(945, 491)

In [8]:
metadados = generate_metadata(data_itub4_clean)
metadados[metadados['qt_nulos'] >=1].head(30)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade


In [9]:
data_itub4_clean3.shape

(945, 491)

In [ ]:
data_itub4.shape

(1686, 27)

In [152]:
Lista_dados_pre_trat = data_itub4_clean3.iloc[:,:-1].columns
Lista_dados_pre_trat
data_all = pd.merge(data_itub4_clean3,data_itub4,how='left')

Lista_dados_pre_trat

joblib.dump(Lista_dados_pre_trat, "Lista_dados_pre_trat.pkl", compress=3)
Lista_dados_pre_trat_charge = joblib.load('/content/Lista_dados_pre_trat.pkl')

data_all.iloc[len(data_all.iloc[:-140,:]):,:].to_parquet("data_validacao.parquet", engine="pyarrow", index=False)
data_validacao = pd.read_parquet("data_validacao.parquet", engine="pyarrow").reset_index(drop=True)

data_all.iloc[:-140,:].to_parquet("data_train_test.parquet", engine="pyarrow", index=False)
data_train_test = pd.read_parquet("data_train_test.parquet", engine="pyarrow").reset_index(drop=True)

In [67]:
# Lista_dados_pre_trat['index','data_pregao','especificacao_papel']




In [165]:
X_train_temp, X_test_temp, Y_train, Y_test = train_test_split(data_train_test[Lista_dados_pre_trat_charge],
                                                    data_train_test['target'],
                                                    test_size=0.2,
                                                    random_state=42)
X_train_temp.shape,X_test_temp.shape,Y_train.shape,Y_test.shape

((644, 490), (161, 490), (644,), (161,))

In [166]:
# FEATURE SELECTION
import lightgbm as lgb

clf = lgb.LGBMRegressor(n_estimators=100, random_state=43, verbose=-1)# troque aqui qual classificador quer utilizar: GradientBoostingClassifier, DecisionTreeClassifier
clf.fit(X_train_temp, Y_train)

# Obter importância das variáveis
feature_importances = clf.feature_importances_
features = pd.DataFrame({
    'Feature': X_train_temp.columns,
    'Importance': feature_importances
})

# Ordenar variáveis por importância
features = features.sort_values(by='Importance', ascending=False)
features


,Feature,Importance
1,numero_negocios,111
0,quantidade_papeis_negociados,93
210,razmedia20_60_quantidade_papeis_negociados,69
240,razmedia20_60_numero_negocios,66
450,razmedia20_60_num_distribuicao_papel,51
...,...,...
343,razmax240_480_preco_minimo,0
174,max120_preco_ultimo_negocio,0
345,razmax60_240preco_minimo,0
173,max60_preco_ultimo_negocio,0


In [167]:
# Estabelecer um limite de corte, por exemplo, x% da importância máxima


selected_features = features['Feature'].iloc[:50].tolist()


joblib.dump(selected_features, "selected_features.pkl", compress=3)
selected_features_charge = joblib.load('/content/selected_features.pkl')

X_train = X_train_temp[selected_features_charge]
X_test = X_test_temp[selected_features_charge]

In [168]:
from hyperopt import fmin, tpe, hp, Trials
from sklearn import metrics

list_estimators = [3,5,7,10, 20, 30, 40, 50]
space = {
    'n_estimators': hp.choice('n_estimators', list_estimators),  # Escolha entre esses valores
    'max_depth': hp.quniform('max_depth', 2, 3, 1),  # Inteiros de 3 a 15
    'min_samples_split': hp.uniform('min_samples_split', 0.1, 1.0),  # Float entre 0.1 e 1.0
    'num_leaves': hp.quniform('num_leaves', 2, 6, 1),  # Inteiros de 2 a 15
}

def objetivo(params):
    modelo = lgb.LGBMRegressor(
        n_estimators=params['n_estimators'],
        max_depth=int(params['max_depth']),
        min_samples_split=params['min_samples_split'],
        num_leaves=int(params['num_leaves']),
        random_state=43
    )

    modelo.fit(X_train, Y_train)

    # Avaliar no conjunto de teste (holdout)
    Y_predi = modelo.predict(X_test)
    score = metrics.mean_absolute_error(Y_test, Y_predi)

    return -score

trials = Trials()  # Salva o histórico das tentativas

best = fmin(
    fn=objetivo,  # Função a minimizar
    space=space,  # Espaço de busca
    algo=tpe.suggest,  # Algoritmo (TPE = Tree-structured Parzen Estimator)
    max_evals=50,  # Número de tentativas
    trials=trials  # Salva histórico
)

print("Melhores Hiperparâmetros:", best)


100%|██████████| 50/50 [00:01<00:00, 32.95trial/s, best loss: -2.2004663685488093]
Melhores Hiperparâmetros: {'max_depth': 2.0, 'min_samples_split': 0.5178508524194141, 'n_estimators': 0, 'num_leaves': 2.0}


In [169]:

modelo_final = lgb.LGBMRegressor(
        n_estimators=list_estimators[best["n_estimators"]],
        max_depth=int(best["max_depth"]),
        min_samples_split=best["min_samples_split"],
        num_leaves=int(best["num_leaves"]),
        random_state=43
)

modelo_final.fit(X_train, Y_train)

joblib.dump(modelo_final, "modelo_final.pkl", compress=3)
modelo_final_charge = joblib.load('/content/modelo_final.pkl')

In [170]:
from sklearn import metrics

X_valid = data_validacao[selected_features_charge]
Y_valid = data_validacao['target']

pred_train = modelo_final_charge.predict(X_train)
pred_test  = modelo_final_charge.predict(X_test)
pred_valid = modelo_final_charge.predict(X_valid)



metrics_df = pd.DataFrame({
    'Conjunto': ['Treino', 'Teste', 'Validação'],
    'média_real': [Y_train.mean(),Y_test.mean(),Y_valid.mean()],
    'média_predita': [pred_train.mean(),pred_test.mean(),pred_valid.mean()],
    'r2': [metrics.r2_score(Y_train, pred_train),metrics.r2_score(Y_test, pred_test),metrics.r2_score(Y_valid, pred_valid)],
    'MAE': [metrics.mean_absolute_error(Y_train, pred_train),metrics.mean_absolute_error(Y_test, pred_test),metrics.mean_absolute_error(Y_valid, pred_valid)],
    'MSE': [metrics.mean_squared_error(Y_train, pred_train),metrics.mean_squared_error(Y_test, pred_test),metrics.mean_squared_error(Y_valid, pred_valid)],
    'RMSE': [np.sqrt(metrics.mean_squared_error(Y_train, pred_train)), np.sqrt(metrics.mean_squared_error(Y_test, pred_test)),np.sqrt(metrics.mean_squared_error(Y_valid, pred_valid))],
    'MAPE': [metrics.mean_absolute_percentage_error(Y_train, pred_train), metrics.mean_absolute_percentage_error(Y_test, pred_test), metrics.mean_absolute_percentage_error(Y_valid, pred_valid)]


    })
metrics_df

,Conjunto,média_real,média_predita,r2,MAE,MSE,RMSE,MAPE
0,Treino,27.368975,27.368975,0.244180,2.096493,7.333866,2.708111,0.076996
1,Teste,27.498385,27.384147,0.231399,2.200466,7.989588,2.826586,0.079370
2,Validação,33.883000,28.078668,-8.970798,5.804332,37.445814,6.119298,0.168648


In [171]:
X_train.columns


Index(['numero_negocios', 'quantidade_papeis_negociados',
       'razmedia20_60_quantidade_papeis_negociados',
       'razmedia20_60_numero_negocios', 'razmedia20_60_num_distribuicao_papel',
       'razmax20_60_numero_negocios', 'preco_maximo',
       'media480_quantidade_papeis_negociados', 'preco_abertura',
       'media240_quantidade_papeis_negociados', 'media240_numero_negocios',
       'razmedia20_120_quantidade_papeis_negociados', 'preco_minimo',
       'media480_numero_negocios', 'razmedia60_480numero_negocios',
       'num_distribuicao_papel', 'razmedia120_480quantidade_papeis_negociados',
       'razmedia20_240quantidade_papeis_negociados',
       'preco_melhor_oferta_venda', 'razmedia120_480numero_negocios',
       'media20_numero_negocios',
       'razmedia60_120_quantidade_papeis_negociados',
       'razmedia20_60_preco_melhor_oferta_venda',
       'razmedia60_480quantidade_papeis_negociados',
       'razmedia20_240num_distribuicao_papel',
       'razmedia20_120_numero_nego

In [184]:
X_train['pred_train'] = pred_train
X_test['pred_test'] = pred_test
X_valid['pred_valid'] = pred_valid

X_train['flag'] = 'treino'
X_test['flag'] = 'teste '
X_valid['flag'] = 'valid '

abt_train = X_train.join(Y_train, how='left')
abt_test = X_train.join(Y_test, how='left')
abt_valid = X_train.join(Y_valid, how='left')




abt_final =  pd.concat([abt_train,abt_test,abt_valid], axis = 0)

<ipython-input-184-cfe2b4dd9d11>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['pred_train'] = pred_train
<ipython-input-184-cfe2b4dd9d11>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['pred_test'] = pred_test
<ipython-input-184-cfe2b4dd9d11>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [186]:
abt_final.to_csv('abt_final.csv', index=False)
